In [2]:
import requests

In [3]:
class Data():
    def __init__(self):
        self.headers = {
            "content-type": "application/json",
            "X-RosetteAPI-Key": "ad9765417caa9e2fd304e0aea0e2d24f",
            "X-RapidAPI-Key": "bc44666f27msh0cea06e275c5146p185e39jsnb306b7888971",
            "X-RapidAPI-Host": "rosetteapi-rosette-v1.p.rapidapi.com"
        }
        self.payload = {}

    def get_token(self, text):
        url = "https://rosetteapi-rosette-v1.p.rapidapi.com/tokens"
        self.payload['content'] = text
        response = requests.request("POST",
                                    url,
                                    json=self.payload,
                                    headers=self.headers)
        return eval(response.text)['tokens']

    def get_lang(self, text):
        url = "https://api.rosette.com/rest/v1/language"
        self.payload['content'] = text
        response = requests.request("POST",
                                    url,
                                    json=self.payload,
                                    headers=self.headers)
        return eval(response.text)["languageDetections"][0]['language']

    def get_vec(self, text):
        url = "https://api.rosette.com/rest/v1/semantics/vector"
        self.payload['content'] = text
        self.payload['options'] = {"perToken": True}
        response = requests.request("POST",
                                    url,
                                    json=self.payload,
                                    headers=self.headers)
        return {
            'documentEmbedding': eval(response.text)['documentEmbedding'],
            'tokenEmbeddings': eval(response.text)['tokenEmbeddings']
        }

In [4]:
def word_wash(text, api_data_class, stop_word='default'):
    #determine language
    if stop_word == 'default':
        lang = api_data_class.get_lang(text)
        if lang == 'zho':
            stop_word = 'pacakge_data/cn_stopwords.txt'
        elif lang == 'eng':
            stop_word = 'pacakge_data/en_stopwords.txt'
        else:
            stop_word = 'pacakge_data/other_stopwords.txt'

    try:
        stop_file = open(stop_word, 'r', encoding='utf-8')
        stopwords = stop_file.read().split("\n")
        stop_file.close()
    except:
        print('{} is not exist, please check the file!'.format(stop_word))

    #check the characters limit (need a function)
    text_lst = check_limit(text)
    text_token = []
    for t in text_lst:
        text_token.extend(api_data_class.get_token(t))

    washed_token = []
    for char in text_token:
        if char in stopwords:
            pass
        else:
            washed_token.append(char)
    return washed_token

def cal_frequency(text, api_data_class, stop_word='default'):
    washed_token = word_wash(text, api_data_class, stop_word)

    counts = {}
    for word in washed_token:
        counts[word] = counts.get(word, 0) + 1

    counts = sorted(counts.items(), key=lambda x: x[1], reverse=True)
    return counts

def check_limit(text):
    text = text.replace('\n', ' ')
    if len(text.replace(' ', '')) <= 5000:
        return [text]
    else:
        true_text = []
        if '.' in text:
            true_text = check_limit_tool(text, '.')
        elif '。' in text:
            true_text = check_limit_tool(text, '。')
        else:
            true_text = check_limit_tool(text, '')
    return true_text


def check_limit_tool(text, symbol):
    temp = []
    string = ''
    if symbol != '':
        split_text = text.split(symbol)
    else:
        split_text = text
    for sen in split_text:
        temp_s = string + sen + symbol
        if len(temp_s.replace(' ', '')) > 5000:
            temp.append(string)
            string = sen + symbol
        else:
            string = string + sen + symbol

        if sen == split_text[-1]:
            temp.append(string)
    return temp

#file_loc: where the wordcloud pic and chart pic should be saved
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt
from altair_saver import save


def word_viz(text,
             file_loc,
             api_data_class,
             top_num,
             stop_word='default',
             cloud_set=WordCloud(font_path='pacakge_data/STKAITI.TTF')):
    
    washed_token = word_wash(text, api_data_class, stop_word)
    
    cloud_set.generate(' '.join(washed_token))
    cloud_set.to_file(file_loc + 'cloud.png')

    counts = cal_frequency(text, api_data_class, stop_word)
    words = []
    count = []
    for i in counts:
        words.append(i[0])
        count.append(i[1])
    df = pd.DataFrame({'words': words, 'count': count})
    chart = (alt.Chart(df[:top_num]).mark_line().encode(
        x='words', y='count').properties(height=400, width=400))
    save(chart, "chart.html")

In [4]:
text = """There was a table set out under a tree in front of the house, and the
March Hare and the Hatter were having tea at it: a Dormouse was sitting
between them, fast asleep, and the other two were using it as a
cushion, resting their elbows on it, and talking over its head. “Very
uncomfortable for the Dormouse,” thought Alice; “only, as it’s asleep,
I suppose it doesn’t mind.”

The table was a large one, but the three were all crowded together at
one corner of it: “No room! No room!” they cried out when they saw
Alice coming. “There’s _plenty_ of room!” said Alice indignantly, and
she sat down in a large arm-chair at one end of the table.

“Have some wine,” the March Hare said in an encouraging tone.

Alice looked all round the table, but there was nothing on it but tea.
“I don’t see any wine,” she remarked.

“There isn’t any,” said the March Hare.

“Then it wasn’t very civil of you to offer it,” said Alice angrily.

“It wasn’t very civil of you to sit down without being invited,” said
the March Hare.

“I didn’t know it was _your_ table,” said Alice; “it’s laid for a great
many more than three.”

“Your hair wants cutting,” said the Hatter. He had been looking at
Alice for some time with great curiosity, and this was his first
speech.

“You should learn not to make personal remarks,” Alice said with some
severity; “it’s very rude.”

The Hatter opened his eyes very wide on hearing this; but all he _said_
was, “Why is a raven like a writing-desk?”

“Come, we shall have some fun now!” thought Alice. “I’m glad they’ve
begun asking riddles.—I believe I can guess that,” she added aloud.

“Do you mean that you think you can find out the answer to it?” said
the March Hare.

“Exactly so,” said Alice.

“Then you should say what you mean,” the March Hare went on.

“I do,” Alice hastily replied; “at least—at least I mean what I
say—that’s the same thing, you know.”

“Not the same thing a bit!” said the Hatter. “You might just as well
say that ‘I see what I eat’ is the same thing as ‘I eat what I see’!”

“You might just as well say,” added the March Hare, “that ‘I like what
I get’ is the same thing as ‘I get what I like’!”

“You might just as well say,” added the Dormouse, who seemed to be
talking in his sleep, “that ‘I breathe when I sleep’ is the same thing
as ‘I sleep when I breathe’!”

“It _is_ the same thing with you,” said the Hatter, and here the
conversation dropped, and the party sat silent for a minute, while
Alice thought over all she could remember about ravens and
writing-desks, which wasn’t much.

The Hatter was the first to break the silence. “What day of the month
is it?” he said, turning to Alice: he had taken his watch out of his
pocket, and was looking at it uneasily, shaking it every now and then,
and holding it to his ear.

Alice considered a little, and then said “The fourth.”

“Two days wrong!” sighed the Hatter. “I told you butter wouldn’t suit
the works!” he added looking angrily at the March Hare.

“It was the _best_ butter,” the March Hare meekly replied.

“Yes, but some crumbs must have got in as well,” the Hatter grumbled:
“you shouldn’t have put it in with the bread-knife.”

The March Hare took the watch and looked at it gloomily: then he dipped
it into his cup of tea, and looked at it again: but he could think of
nothing better to say than his first remark, “It was the _best_ butter,
you know.”

Alice had been looking over his shoulder with some curiosity. “What a
funny watch!” she remarked. “It tells the day of the month, and doesn’t
tell what o’clock it is!”

“Why should it?” muttered the Hatter. “Does _your_ watch tell you what
year it is?”

“Of course not,” Alice replied very readily: “but that’s because it
stays the same year for such a long time together.”

“Which is just the case with _mine_,” said the Hatter.

Alice felt dreadfully puzzled. The Hatter’s remark seemed to have no
sort of meaning in it, and yet it was certainly English. “I don’t quite
understand you,” she said, as politely as she could.

“The Dormouse is asleep again,” said the Hatter, and he poured a little
hot tea upon its nose.

The Dormouse shook its head impatiently, and said, without opening its
eyes, “Of course, of course; just what I was going to remark myself.”

“Have you guessed the riddle yet?” the Hatter said, turning to Alice
again.

“No, I give it up,” Alice replied: “what’s the answer?”

“I haven’t the slightest idea,” said the Hatter.

“Nor I,” said the March Hare.

Alice sighed wearily. “I think you might do something better with the
time,” she said, “than waste it in asking riddles that have no
answers.”

“If you knew Time as well as I do,” said the Hatter, “you wouldn’t talk
about wasting _it_. It’s _him_.”

“I don’t know what you mean,” said Alice.

“Of course you don’t!” the Hatter said, tossing his head
contemptuously. “I dare say you never even spoke to Time!”

“Perhaps not,” Alice cautiously replied: “but I know I have to beat
time when I learn music.”

“Ah! that accounts for it,” said the Hatter. “He won’t stand beating.
Now, if you only kept on good terms with him, he’d do almost anything
you liked with the clock. For instance, suppose it were nine o’clock in
the morning, just time to begin lessons: you’d only have to whisper a
hint to Time, and round goes the clock in a twinkling! Half-past one,
time for dinner!”

(“I only wish it was,” the March Hare said to itself in a whisper.)

“That would be grand, certainly,” said Alice thoughtfully: “but then—I
shouldn’t be hungry for it, you know.”

“Not at first, perhaps,” said the Hatter: “but you could keep it to
half-past one as long as you liked.”

“Is that the way _you_ manage?” Alice asked.

The Hatter shook his head mournfully. “Not I!” he replied. “We
quarrelled last March—just before _he_ went mad, you know—” (pointing
with his tea spoon at the March Hare,) “—it was at the great concert
given by the Queen of Hearts, and I had to sing

‘Twinkle, twinkle, little bat!
How I wonder what you’re at!’


You know the song, perhaps?”

“I’ve heard something like it,” said Alice.

“It goes on, you know,” the Hatter continued, “in this way:—

‘Up above the world you fly,
Like a tea-tray in the sky.
                    Twinkle, twinkle—’”


Here the Dormouse shook itself, and began singing in its sleep
“_Twinkle, twinkle, twinkle, twinkle_—” and went on so long that they
had to pinch it to make it stop.

“Well, I’d hardly finished the first verse,” said the Hatter, “when the
Queen jumped up and bawled out, ‘He’s murdering the time! Off with his
head!’”

“How dreadfully savage!” exclaimed Alice.

“And ever since that,” the Hatter went on in a mournful tone, “he won’t
do a thing I ask! It’s always six o’clock now.”

A bright idea came into Alice’s head. “Is that the reason so many
tea-things are put out here?” she asked.

“Yes, that’s it,” said the Hatter with a sigh: “it’s always tea-time,
and we’ve no time to wash the things between whiles.”

“Then you keep moving round, I suppose?” said Alice.

“Exactly so,” said the Hatter: “as the things get used up.”

“But what happens when you come to the beginning again?” Alice ventured
to ask.

“Suppose we change the subject,” the March Hare interrupted, yawning.
“I’m getting tired of this. I vote the young lady tells us a story.”

“I’m afraid I don’t know one,” said Alice, rather alarmed at the
proposal.

“Then the Dormouse shall!” they both cried. “Wake up, Dormouse!” And
they pinched it on both sides at once.

The Dormouse slowly opened his eyes. “I wasn’t asleep,” he said in a
hoarse, feeble voice: “I heard every word you fellows were saying.”

“Tell us a story!” said the March Hare.

“Yes, please do!” pleaded Alice.

“And be quick about it,” added the Hatter, “or you’ll be asleep again
before it’s done.”

“Once upon a time there were three little sisters,” the Dormouse began
in a great hurry; “and their names were Elsie, Lacie, and Tillie; and
they lived at the bottom of a well—”

“What did they live on?” said Alice, who always took a great interest
in questions of eating and drinking.

“They lived on treacle,” said the Dormouse, after thinking a minute or
two.

“They couldn’t have done that, you know,” Alice gently remarked;
“they’d have been ill.”

“So they were,” said the Dormouse; “_very_ ill.”

Alice tried to fancy to herself what such an extraordinary ways of
living would be like, but it puzzled her too much, so she went on: “But
why did they live at the bottom of a well?”

“Take some more tea,” the March Hare said to Alice, very earnestly.

“I’ve had nothing yet,” Alice replied in an offended tone, “so I can’t
take more.”

“You mean you can’t take _less_,” said the Hatter: “it’s very easy to
take _more_ than nothing.”

“Nobody asked _your_ opinion,” said Alice.

“Who’s making personal remarks now?” the Hatter asked triumphantly.

Alice did not quite know what to say to this: so she helped herself to
some tea and bread-and-butter, and then turned to the Dormouse, and
repeated her question. “Why did they live at the bottom of a well?”

The Dormouse again took a minute or two to think about it, and then
said, “It was a treacle-well.”

“There’s no such thing!” Alice was beginning very angrily, but the
Hatter and the March Hare went “Sh! sh!” and the Dormouse sulkily
remarked, “If you can’t be civil, you’d better finish the story for
yourself.”

“No, please go on!” Alice said very humbly; “I won’t interrupt again. I
dare say there may be _one_.”

“One, indeed!” said the Dormouse indignantly. However, he consented to
go on. “And so these three little sisters—they were learning to draw,
you know—”

“What did they draw?” said Alice, quite forgetting her promise.

“Treacle,” said the Dormouse, without considering at all this time.

“I want a clean cup,” interrupted the Hatter: “let’s all move one place
on.”

He moved on as he spoke, and the Dormouse followed him: the March Hare
moved into the Dormouse’s place, and Alice rather unwillingly took the
place of the March Hare. The Hatter was the only one who got any
advantage from the change: and Alice was a good deal worse off than
before, as the March Hare had just upset the milk-jug into his plate.

Alice did not wish to offend the Dormouse again, so she began very
cautiously: “But I don’t understand. Where did they draw the treacle
from?”

“You can draw water out of a water-well,” said the Hatter; “so I should
think you could draw treacle out of a treacle-well—eh, stupid?”

“But they were _in_ the well,” Alice said to the Dormouse, not choosing
to notice this last remark.

“Of course they were,” said the Dormouse; “—well in.”

This answer so confused poor Alice, that she let the Dormouse go on for
some time without interrupting it.

“They were learning to draw,” the Dormouse went on, yawning and rubbing
its eyes, for it was getting very sleepy; “and they drew all manner of
things—everything that begins with an M—”

“Why with an M?” said Alice.

“Why not?” said the March Hare.

Alice was silent.

The Dormouse had closed its eyes by this time, and was going off into a
doze; but, on being pinched by the Hatter, it woke up again with a
little shriek, and went on: “—that begins with an M, such as
mouse-traps, and the moon, and memory, and muchness—you know you say
things are “much of a muchness”—did you ever see such a thing as a
drawing of a muchness?”

“Really, now you ask me,” said Alice, very much confused, “I don’t
think—”

“Then you shouldn’t talk,” said the Hatter.

This piece of rudeness was more than Alice could bear: she got up in
great disgust, and walked off; the Dormouse fell asleep instantly, and
neither of the others took the least notice of her going, though she
looked back once or twice, half hoping that they would call after her:
the last time she saw them, they were trying to put the Dormouse into
the teapot.

“At any rate I’ll never go _there_ again!” said Alice as she picked her
way through the wood. “It’s the stupidest tea-party I ever was at in
all my life!”

Just as she said this, she noticed that one of the trees had a door
leading right into it. “That’s very curious!” she thought. “But
everything’s curious today. I think I may as well go in at once.” And
in she went.

Once more she found herself in the long hall, and close to the little
glass table. “Now, I’ll manage better this time,” she said to herself,
and began by taking the little golden key, and unlocking the door that
led into the garden. Then she went to work nibbling at the mushroom
(she had kept a piece of it in her pocket) till she was about a foot
high: then she walked down the little passage: and _then_—she found
herself at last in the beautiful garden, among the bright flower-beds
and the cool fountains.




CHAPTER VIII.
The Queen’s Croquet-Ground


A large rose-tree stood near the entrance of the garden: the roses
growing on it were white, but there were three gardeners at it, busily
painting them red. Alice thought this a very curious thing, and she
went nearer to watch them, and just as she came up to them she heard
one of them say, “Look out now, Five! Don’t go splashing paint over me
like that!”

“I couldn’t help it,” said Five, in a sulky tone; “Seven jogged my
elbow.”

On which Seven looked up and said, “That’s right, Five! Always lay the
blame on others!”

“_You’d_ better not talk!” said Five. “I heard the Queen say only
yesterday you deserved to be beheaded!”

“What for?” said the one who had spoken first.

“That’s none of _your_ business, Two!” said Seven.

“Yes, it _is_ his business!” said Five, “and I’ll tell him—it was for
bringing the cook tulip-roots instead of onions.”

Seven flung down his brush, and had just begun “Well, of all the unjust
things—” when his eye chanced to fall upon Alice, as she stood watching
them, and he checked himself suddenly: the others looked round also,
and all of them bowed low.

“Would you tell me,” said Alice, a little timidly, “why you are
painting those roses?”

Five and Seven said nothing, but looked at Two. Two began in a low
voice, “Why the fact is, you see, Miss, this here ought to have been a
_red_ rose-tree, and we put a white one in by mistake; and if the Queen
was to find it out, we should all have our heads cut off, you know. So
you see, Miss, we’re doing our best, afore she comes, to—” At this
moment Five, who had been anxiously looking across the garden, called
out “The Queen! The Queen!” and the three gardeners instantly threw
themselves flat upon their faces. There was a sound of many footsteps,
and Alice looked round, eager to see the Queen.

First came ten soldiers carrying clubs; these were all shaped like the
three gardeners, oblong and flat, with their hands and feet at the
corners: next the ten courtiers; these were ornamented all over with
diamonds, and walked two and two, as the soldiers did. After these came
the royal children; there were ten of them, and the little dears came
jumping merrily along hand in hand, in couples: they were all
ornamented with hearts. Next came the guests, mostly Kings and Queens,
and among them Alice recognised the White Rabbit: it was talking in a
hurried nervous manner, smiling at everything that was said, and went
by without noticing her. Then followed the Knave of Hearts, carrying
the King’s crown on a crimson velvet cushion; and, last of all this
grand procession, came THE KING AND QUEEN OF HEARTS.

Alice was rather doubtful whether she ought not to lie down on her face
like the three gardeners, but she could not remember ever having heard
of such a rule at processions; “and besides, what would be the use of a
procession,” thought she, “if people had all to lie down upon their
faces, so that they couldn’t see it?” So she stood still where she was,
and waited.

When the procession came opposite to Alice, they all stopped and looked
at her, and the Queen said severely “Who is this?” She said it to the
Knave of Hearts, who only bowed and smiled in reply.

“Idiot!” said the Queen, tossing her head impatiently; and, turning to
Alice, she went on, “What’s your name, child?”

“My name is Alice, so please your Majesty,” said Alice very politely;
but she added, to herself, “Why, they’re only a pack of cards, after
all. I needn’t be afraid of them!”

“And who are _these?_” said the Queen, pointing to the three gardeners
who were lying round the rose-tree; for, you see, as they were lying on
their faces, and the pattern on their backs was the same as the rest of
the pack, she could not tell whether they were gardeners, or soldiers,
or courtiers, or three of her own children.

“How should _I_ know?” said Alice, surprised at her own courage. “It’s
no business of _mine_.”

The Queen turned crimson with fury, and, after glaring at her for a
moment like a wild beast, screamed “Off with her head! Off—”

“Nonsense!” said Alice, very loudly and decidedly, and the Queen was
silent.

The King laid his hand upon her arm, and timidly said “Consider, my
dear: she is only a child!”

The Queen turned angrily away from him, and said to the Knave “Turn
them over!”

The Knave did so, very carefully, with one foot.

“Get up!” said the Queen, in a shrill, loud voice, and the three
gardeners instantly jumped up, and began bowing to the King, the Queen,
the royal children, and everybody else.

“Leave off that!” screamed the Queen. “You make me giddy.” And then,
turning to the rose-tree, she went on, “What _have_ you been doing
here?”

“May it please your Majesty,” said Two, in a very humble tone, going
down on one knee as he spoke, “we were trying—”

“_I_ see!” said the Queen, who had meanwhile been examining the roses.
“Off with their heads!” and the procession moved on, three of the
soldiers remaining behind to execute the unfortunate gardeners, who ran
to Alice for protection.

“You shan’t be beheaded!” said Alice, and she put them into a large
flower-pot that stood near. The three soldiers wandered about for a
minute or two, looking for them, and then quietly marched off after the
others.

“Are their heads off?” shouted the Queen.

“Their heads are gone, if it please your Majesty!” the soldiers shouted
in reply.

“That’s right!” shouted the Queen. “Can you play croquet?”

The soldiers were silent, and looked at Alice, as the question was
evidently meant for her.

“Yes!” shouted Alice.

“Come on, then!” roared the Queen, and Alice joined the procession,
wondering very much what would happen next.

“It’s—it’s a very fine day!” said a timid voice at her side. She was
walking by the White Rabbit, who was peeping anxiously into her face.

“Very,” said Alice: “—where’s the Duchess?”

“Hush! Hush!” said the Rabbit in a low, hurried tone. He looked
anxiously over his shoulder as he spoke, and then raised himself upon
tiptoe, put his mouth close to her ear, and whispered “She’s under
sentence of execution.”

“What for?” said Alice.

“Did you say ‘What a pity!’?” the Rabbit asked.

“No, I didn’t,” said Alice: “I don’t think it’s at all a pity. I said
‘What for?’”

“She boxed the Queen’s ears—” the Rabbit began. Alice gave a little
scream of laughter. “Oh, hush!” the Rabbit whispered in a frightened
tone. “The Queen will hear you! You see, she came rather late, and the
Queen said—”

“Get to your places!” shouted the Queen in a voice of thunder, and
people began running about in all directions, tumbling up against each
other; however, they got settled down in a minute or two, and the game
began. Alice thought she had never seen such a curious croquet-ground
in her life; it was all ridges and furrows; the balls were live
hedgehogs, the mallets live flamingoes, and the soldiers had to double
themselves up and to stand on their hands and feet, to make the arches.

The chief difficulty Alice found at first was in managing her flamingo:
she succeeded in getting its body tucked away, comfortably enough,
under her arm, with its legs hanging down, but generally, just as she
had got its neck nicely straightened out, and was going to give the
hedgehog a blow with its head, it _would_ twist itself round and look
up in her face, with such a puzzled expression that she could not help
bursting out laughing: and when she had got its head down, and was
going to begin again, it was very provoking to find that the hedgehog
had unrolled itself, and was in the act of crawling away: besides all
this, there was generally a ridge or furrow in the way wherever she
wanted to send the hedgehog to, and, as the doubled-up soldiers were
always getting up and walking off to other parts of the ground, Alice
soon came to the conclusion that it was a very difficult game indeed.

The players all played at once without waiting for turns, quarrelling
all the while, and fighting for the hedgehogs; and in a very short time
the Queen was in a furious passion, and went stamping about, and
shouting “Off with his head!” or “Off with her head!” about once in a
minute.

Alice began to feel very uneasy: to be sure, she had not as yet had any
dispute with the Queen, but she knew that it might happen any minute,
“and then,” thought she, “what would become of me? They’re dreadfully
fond of beheading people here; the great wonder is, that there’s any
one left alive!”

She was looking about for some way of escape, and wondering whether she
could get away without being seen, when she noticed a curious
appearance in the air: it puzzled her very much at first, but, after
watching it a minute or two, she made it out to be a grin, and she said
to herself “It’s the Cheshire Cat: now I shall have somebody to talk
to.”

“How are you getting on?” said the Cat, as soon as there was mouth
enough for it to speak with.

Alice waited till the eyes appeared, and then nodded. “It’s no use
speaking to it,” she thought, “till its ears have come, or at least one
of them.” In another minute the whole head appeared, and then Alice put
down her flamingo, and began an account of the game, feeling very glad
she had someone to listen to her. The Cat seemed to think that there
was enough of it now in sight, and no more of it appeared.

“I don’t think they play at all fairly,” Alice began, in rather a
complaining tone, “and they all quarrel so dreadfully one can’t hear
oneself speak—and they don’t seem to have any rules in particular; at
least, if there are, nobody attends to them—and you’ve no idea how
confusing it is all the things being alive; for instance, there’s the
arch I’ve got to go through next walking about at the other end of the
ground—and I should have croqueted the Queen’s hedgehog just now, only
it ran away when it saw mine coming!”

“How do you like the Queen?” said the Cat in a low voice.

“Not at all,” said Alice: “she’s so extremely—” Just then she noticed
that the Queen was close behind her, listening: so she went on,
“—likely to win, that it’s hardly worth while finishing the game.”

The Queen smiled and passed on.

“Who _are_ you talking to?” said the King, going up to Alice, and
looking at the Cat’s head with great curiosity.

“It’s a friend of mine—a Cheshire Cat,” said Alice: “allow me to
introduce it.”

“I don’t like the look of it at all,” said the King: “however, it may
kiss my hand if it likes.”

“I’d rather not,” the Cat remarked.

“Don’t be impertinent,” said the King, “and don’t look at me like
that!” He got behind Alice as he spoke.

“A cat may look at a king,” said Alice. “I’ve read that in some book,
but I don’t remember where.”

“Well, it must be removed,” said the King very decidedly, and he called
the Queen, who was passing at the moment, “My dear! I wish you would
have this cat removed!”

The Queen had only one way of settling all difficulties, great or
small. “Off with his head!” she said, without even looking round.

“I’ll fetch the executioner myself,” said the King eagerly, and he
hurried off.

Alice thought she might as well go back, and see how the game was going
on, as she heard the Queen’s voice in the distance, screaming with
passion. She had already heard her sentence three of the players to be
executed for having missed their turns, and she did not like the look
of things at all, as the game was in such confusion that she never knew
whether it was her turn or not. So she went in search of her hedgehog.

The hedgehog was engaged in a fight with another hedgehog, which seemed
to Alice an excellent opportunity for croqueting one of them with the
other: the only difficulty was, that her flamingo was gone across to
the other side of the garden, where Alice could see it trying in a
helpless sort of way to fly up into a tree.

By the time she had caught the flamingo and brought it back, the fight
was over, and both the hedgehogs were out of sight: “but it doesn’t
matter much,” thought Alice, “as all the arches are gone from this side
of the ground.” So she tucked it away under her arm, that it might not
escape again, and went back for a little more conversation with her
friend.

When she got back to the Cheshire Cat, she was surprised to find quite
a large crowd collected round it: there was a dispute going on between
the executioner, the King, and the Queen, who were all talking at once,
while all the rest were quite silent, and looked very uncomfortable.

The moment Alice appeared, she was appealed to by all three to settle
the question, and they repeated their arguments to her, though, as they
all spoke at once, she found it very hard indeed to make out exactly
what they said.

The executioner’s argument was, that you couldn’t cut off a head unless
there was a body to cut it off from: that he had never had to do such a
thing before, and he wasn’t going to begin at _his_ time of life.

The King’s argument was, that anything that had a head could be
beheaded, and that you weren’t to talk nonsense.

The Queen’s argument was, that if something wasn’t done about it in
less than no time she’d have everybody executed, all round. (It was
this last remark that had made the whole party look so grave and
anxious.)

Alice could think of nothing else to say but “It belongs to the
Duchess: you’d better ask _her_ about it.”

“She’s in prison,” the Queen said to the executioner: “fetch her here.”
And the executioner went off like an arrow.

The Cat’s head began fading away the moment he was gone, and, by the
time he had come back with the Duchess, it had entirely disappeared; so
the King and the executioner ran wildly up and down looking for it,
while the rest of the party went back to the game.




CHAPTER IX.
The Mock Turtle’s Story


“You can’t think how glad I am to see you again, you dear old thing!”
said the Duchess, as she tucked her arm affectionately into Alice’s,
and they walked off together.

Alice was very glad to find her in such a pleasant temper, and thought
to herself that perhaps it was only the pepper that had made her so
savage when they met in the kitchen.

“When _I’m_ a Duchess,” she said to herself, (not in a very hopeful
tone though), “I won’t have any pepper in my kitchen _at all_. Soup
does very well without—Maybe it’s always pepper that makes people
hot-tempered,” she went on, very much pleased at having found out a new
kind of rule, “and vinegar that makes them sour—and camomile that makes
them bitter—and—and barley-sugar and such things that make children
sweet-tempered. I only wish people knew _that_: then they wouldn’t be
so stingy about it, you know—”

She had quite forgotten the Duchess by this time, and was a little
startled when she heard her voice close to her ear. “You’re thinking
about something, my dear, and that makes you forget to talk. I can’t
tell you just now what the moral of that is, but I shall remember it in
a bit.”

“Perhaps it hasn’t one,” Alice ventured to remark.

“Tut, tut, child!” said the Duchess. “Everything’s got a moral, if only
you can find it.” And she squeezed herself up closer to Alice’s side as
she spoke.

Alice did not much like keeping so close to her: first, because the
Duchess was _very_ ugly; and secondly, because she was exactly the
right height to rest her chin upon Alice’s shoulder, and it was an
uncomfortably sharp chin. However, she did not like to be rude, so she
bore it as well as she could.

“The game’s going on rather better now,” she said, by way of keeping up
the conversation a little.

“’Tis so,” said the Duchess: “and the moral of that is—‘Oh, ’tis love,
’tis love, that makes the world go round!’”

“Somebody said,” Alice whispered, “that it’s done by everybody minding
their own business!”

“Ah, well! It means much the same thing,” said the Duchess, digging her
sharp little chin into Alice’s shoulder as she added, “and the moral of
_that_ is—‘Take care of the sense, and the sounds will take care of
themselves.’”

“How fond she is of finding morals in things!” Alice thought to
herself.

“I dare say you’re wondering why I don’t put my arm round your waist,”
the Duchess said after a pause: “the reason is, that I’m doubtful about
the temper of your flamingo. Shall I try the experiment?”

“He might bite,” Alice cautiously replied, not feeling at all anxious
to have the experiment tried.

“Very true,” said the Duchess: “flamingoes and mustard both bite. And
the moral of that is—‘Birds of a feather flock together.’”

“Only mustard isn’t a bird,” Alice remarked.

“Right, as usual,” said the Duchess: “what a clear way you have of
putting things!”

“It’s a mineral, I _think_,” said Alice.

“Of course it is,” said the Duchess, who seemed ready to agree to
everything that Alice said; “there’s a large mustard-mine near here.
And the moral of that is—‘The more there is of mine, the less there is
of yours.’”

“Oh, I know!” exclaimed Alice, who had not attended to this last
remark, “it’s a vegetable. It doesn’t look like one, but it is.”

“I quite agree with you,” said the Duchess; “and the moral of that
is—‘Be what you would seem to be’—or if you’d like it put more
simply—‘Never imagine yourself not to be otherwise than what it might
appear to others that what you were or might have been was not
otherwise than what you had been would have appeared to them to be
otherwise.’”

“I think I should understand that better,” Alice said very politely,
“if I had it written down: but I can’t quite follow it as you say it.”

“That’s nothing to what I could say if I chose,” the Duchess replied,
in a pleased tone.

“Pray don’t trouble yourself to say it any longer than that,” said
Alice.

“Oh, don’t talk about trouble!” said the Duchess. “I make you a present
of everything I’ve said as yet.”

“A cheap sort of present!” thought Alice. “I’m glad they don’t give
birthday presents like that!” But she did not venture to say it out
loud.

“Thinking again?” the Duchess asked, with another dig of her sharp
little chin.

“I’ve a right to think,” said Alice sharply, for she was beginning to
feel a little worried.

“Just about as much right,” said the Duchess, “as pigs have to fly; and
the m—”

But here, to Alice’s great surprise, the Duchess’s voice died away,
even in the middle of her favourite word ‘moral,’ and the arm that was
linked into hers began to tremble. Alice looked up, and there stood the
Queen in front of them, with her arms folded, frowning like a
thunderstorm.

“A fine day, your Majesty!” the Duchess began in a low, weak voice.

“Now, I give you fair warning,” shouted the Queen, stamping on the
ground as she spoke; “either you or your head must be off, and that in
about half no time! Take your choice!”

The Duchess took her choice, and was gone in a moment.

“Let’s go on with the game,” the Queen said to Alice; and Alice was too
much frightened to say a word, but slowly followed her back to the
croquet-ground.

The other guests had taken advantage of the Queen’s absence, and were
resting in the shade: however, the moment they saw her, they hurried
back to the game, the Queen merely remarking that a moment’s delay
would cost them their lives.

All the time they were playing the Queen never left off quarrelling
with the other players, and shouting “Off with his head!” or “Off with
her head!” Those whom she sentenced were taken into custody by the
soldiers, who of course had to leave off being arches to do this, so
that by the end of half an hour or so there were no arches left, and
all the players, except the King, the Queen, and Alice, were in custody
and under sentence of execution.

Then the Queen left off, quite out of breath, and said to Alice, “Have
you seen the Mock Turtle yet?”

“No,” said Alice. “I don’t even know what a Mock Turtle is.”

“It’s the thing Mock Turtle Soup is made from,” said the Queen.

“I never saw one, or heard of one,” said Alice.

“Come on, then,” said the Queen, “and he shall tell you his history,”

As they walked off together, Alice heard the King say in a low voice,
to the company generally, “You are all pardoned.” “Come, _that’s_ a
good thing!” she said to herself, for she had felt quite unhappy at the
number of executions the Queen had ordered.

They very soon came upon a Gryphon, lying fast asleep in the sun. (If
you don’t know what a Gryphon is, look at the picture.) “Up, lazy
thing!” said the Queen, “and take this young lady to see the Mock
Turtle, and to hear his history. I must go back and see after some
executions I have ordered;” and she walked off, leaving Alice alone
with the Gryphon. Alice did not quite like the look of the creature,
but on the whole she thought it would be quite as safe to stay with it
as to go after that savage Queen: so she waited.

The Gryphon sat up and rubbed its eyes: then it watched the Queen till
she was out of sight: then it chuckled. “What fun!” said the Gryphon,
half to itself, half to Alice.

“What _is_ the fun?” said Alice.

“Why, _she_,” said the Gryphon. “It’s all her fancy, that: they never
executes nobody, you know. Come on!”

“Everybody says ‘come on!’ here,” thought Alice, as she went slowly
after it: “I never was so ordered about in all my life, never!”

They had not gone far before they saw the Mock Turtle in the distance,
sitting sad and lonely on a little ledge of rock, and, as they came
nearer, Alice could hear him sighing as if his heart would break. She
pitied him deeply. “What is his sorrow?” she asked the Gryphon, and the
Gryphon answered, very nearly in the same words as before, “It’s all
his fancy, that: he hasn’t got no sorrow, you know. Come on!”

So they went up to the Mock Turtle, who looked at them with large eyes
full of tears, but said nothing.

“This here young lady,” said the Gryphon, “she wants for to know your
history, she do.”

“I’ll tell it her,” said the Mock Turtle in a deep, hollow tone: “sit
down, both of you, and don’t speak a word till I’ve finished.”

So they sat down, and nobody spoke for some minutes. Alice thought to
herself, “I don’t see how he can _ever_ finish, if he doesn’t begin.”
But she waited patiently.

“Once,” said the Mock Turtle at last, with a deep sigh, “I was a real
Turtle.”

These words were followed by a very long silence, broken only by an
occasional exclamation of “Hjckrrh!” from the Gryphon, and the constant
heavy sobbing of the Mock Turtle. Alice was very nearly getting up and
saying, “Thank you, sir, for your interesting story,” but she could not
help thinking there _must_ be more to come, so she sat still and said
nothing.

“When we were little,” the Mock Turtle went on at last, more calmly,
though still sobbing a little now and then, “we went to school in the
sea. The master was an old Turtle—we used to call him Tortoise—”

“Why did you call him Tortoise, if he wasn’t one?” Alice asked.

“We called him Tortoise because he taught us,” said the Mock Turtle
angrily: “really you are very dull!”

“You ought to be ashamed of yourself for asking such a simple
question,” added the Gryphon; and then they both sat silent and looked
at poor Alice, who felt ready to sink into the earth. At last the
Gryphon said to the Mock Turtle, “Drive on, old fellow! Don’t be all
day about it!” and he went on in these words:

“Yes, we went to school in the sea, though you mayn’t believe it—”

“I never said I didn’t!” interrupted Alice.

“You did,” said the Mock Turtle.

“Hold your tongue!” added the Gryphon, before Alice could speak again.
The Mock Turtle went on.

“We had the best of educations—in fact, we went to school every day—”

“_I’ve_ been to a day-school, too,” said Alice; “you needn’t be so
proud as all that.”

“With extras?” asked the Mock Turtle a little anxiously.

“Yes,” said Alice, “we learned French and music.”

“And washing?” said the Mock Turtle.

“Certainly not!” said Alice indignantly.

“Ah! then yours wasn’t a really good school,” said the Mock Turtle in a
tone of great relief. “Now at _ours_ they had at the end of the bill,
‘French, music, _and washing_—extra.’”

“You couldn’t have wanted it much,” said Alice; “living at the bottom
of the sea.”

“I couldn’t afford to learn it.” said the Mock Turtle with a sigh. “I
only took the regular course.”

“What was that?” inquired Alice.

“Reeling and Writhing, of course, to begin with,” the Mock Turtle
replied; “and then the different branches of Arithmetic—Ambition,
Distraction, Uglification, and Derision.”

“I never heard of ‘Uglification,’” Alice ventured to say. “What is it?”

The Gryphon lifted up both its paws in surprise. “What! Never heard of
uglifying!” it exclaimed. “You know what to beautify is, I suppose?”

“Yes,” said Alice doubtfully: “it means—to—make—anything—prettier.”

“Well, then,” the Gryphon went on, “if you don’t know what to uglify
is, you _are_ a simpleton.”

Alice did not feel encouraged to ask any more questions about it, so
she turned to the Mock Turtle, and said “What else had you to learn?”

“Well, there was Mystery,” the Mock Turtle replied, counting off the
subjects on his flappers, “—Mystery, ancient and modern, with
Seaography: then Drawling—the Drawling-master was an old conger-eel,
that used to come once a week: _he_ taught us Drawling, Stretching, and
Fainting in Coils.”"""

In [8]:
api_data = Data()

In [14]:
#test 1
word_wash(text, api_data, stop_word='default')

['There',
 'table',
 'set',
 'tree',
 'front',
 'house',
 'March',
 'Hare',
 'Hatter',
 'tea',
 'Dormouse',
 'sitting',
 'fast',
 'asleep',
 'cushion',
 'resting',
 'elbows',
 'talking',
 'head',
 'Very',
 'uncomfortable',
 'Dormouse',
 'Alice',
 'asleep',
 'suppose',
 'doesn’t',
 'mind',
 'The',
 'table',
 'crowded',
 'corner',
 'No',
 'No',
 'cried',
 'Alice',
 'coming',
 'There',
 'plenty',
 'Alice',
 'indignantly',
 'sat',
 'arm',
 'chair',
 'table',
 'Have',
 'wine',
 'March',
 'Hare',
 'encouraging',
 'tone',
 'Alice',
 'looked',
 'round',
 'table',
 'tea',
 'don’t',
 'wine',
 'remarked',
 'There',
 'isn’t',
 'March',
 'Hare',
 'Then',
 'wasn’t',
 'civil',
 'offer',
 'Alice',
 'angrily',
 'It',
 'wasn’t',
 'civil',
 'sit',
 'invited',
 'March',
 'Hare',
 'didn’t',
 'table',
 'Alice',
 'laid',
 'Your',
 'hair',
 'cutting',
 'Hatter',
 'He',
 'Alice',
 'time',
 'curiosity',
 'speech',
 'You',
 'learn',
 'personal',
 'remarks',
 'Alice',
 'severity',
 'rude',
 'The',
 'Hatter',
 'ey

In [16]:
#test 2
cal_frequency(text, api_data, stop_word='default')

[('Alice', 137),
 ('Queen', 51),
 ('The', 43),
 ('Hatter', 33),
 ('Dormouse', 27),
 ('Turtle', 24),
 ('It', 23),
 ('time', 23),
 ('March', 22),
 ('don’t', 22),
 ('Duchess', 22),
 ('Mock', 22),
 ('Hare', 21),
 ('head', 21),
 ('You', 18),
 ('What', 17),
 ('looked', 15),
 ('Gryphon', 15),
 ('King', 13),
 ('tone', 12),
 ('round', 12),
 ('heard', 12),
 ('And', 12),
 ('tea', 11),
 ('replied', 11),
 ('voice', 11),
 ('soldiers', 10),
 ('game', 10),
 ('Why', 9),
 ('minute', 9),
 ('spoke', 9),
 ('Off', 9),
 ('She', 9),
 ('Cat', 9),
 ('Then', 8),
 ('wasn’t', 8),
 ('eyes', 8),
 ('Yes', 8),
 ('But', 8),
 ('gardeners', 8),
 ('moment', 8),
 ('moral', 8),
 ('asleep', 7),
 ('arm', 7),
 ('He', 7),
 ('day', 7),
 ('remark', 7),
 ('talk', 7),
 ('So', 7),
 ('can’t', 7),
 ('Five', 7),
 ('hedgehog', 7),
 ('table', 6),
 ('tree', 6),
 ('No', 6),
 ('sat', 6),
 ('remarked', 6),
 ('Come', 6),
 ('silent', 6),
 ('mine', 6),
 ('That', 6),
 ('half', 6),
 ('How', 6),
 ('They', 6),
 ('couldn’t', 6),
 ('draw', 6),
 ('wal

In [19]:
#test 3
check_limit(text)

['There was a table set out under a tree in front of the house, and the March Hare and the Hatter were having tea at it: a Dormouse was sitting between them, fast asleep, and the other two were using it as a cushion, resting their elbows on it, and talking over its head. “Very uncomfortable for the Dormouse,” thought Alice; “only, as it’s asleep, I suppose it doesn’t mind.”  The table was a large one, but the three were all crowded together at one corner of it: “No room! No room!” they cried out when they saw Alice coming. “There’s _plenty_ of room!” said Alice indignantly, and she sat down in a large arm-chair at one end of the table.  “Have some wine,” the March Hare said in an encouraging tone.  Alice looked all round the table, but there was nothing on it but tea. “I don’t see any wine,” she remarked.  “There isn’t any,” said the March Hare.  “Then it wasn’t very civil of you to offer it,” said Alice angrily.  “It wasn’t very civil of you to sit down without being invited,” said th

In [46]:
#test 4
word_viz(text,
             '',
             api_data,
             15,
             stop_word='default')

In [23]:
text_zh = """却说陈宫临欲下手杀曹操，忽转念曰：“我为国家跟他到此，杀之不义。不若弃而他
往。”插剑上马，不等天明，自投东郡去了。操觉，不见陈宫，寻思：“此人见我说了这两
句，疑我不仁，弃我而去；吾当急行，不可久留。”遂连夜到陈留，寻见父亲，备说前事；
欲散家资，招募义兵。父言：“资少恐不成事。此间有孝廉卫弘，疏财仗义，其家巨富；若
得相助，事可图矣。”操置酒张筵，拜请卫弘到家，告曰：“今汉室无主，董卓专权，欺君
害民，天下切齿。操欲力扶社稷，恨力不足。公乃忠义之士，敢求相助！”卫弘曰：“吾有
是心久矣，恨未遇英雄耳。既孟德有大志，愿将家资相助。”操大喜；于是先发矫诏，驰报
各道，然后招集义兵，竖起招兵白旗一面，上书“忠义”二字。不数日间，应募之士，如雨
骈集。

    一日，有一个阳平卫国人，姓乐，名进，字文谦，来投曹操。又有一个山阳巨鹿人，姓
李，名典，字曼成，也来投曹操。操皆留为帐前吏。又有沛国谯人夏侯惇，字元让，乃夏侯
婴之后；自小习枪棒；年十四从师学武，有人辱骂其师，惇杀之，逃于外方；闻知曹操起
兵，与其族弟夏侯渊两个，各引壮士千人来会。此二人本操之弟兄：操父曹嵩原是夏侯氏之
子，过房与曹家，因此是同族。不数日，曹氏兄弟曹仁、曹洪各引兵千余来助。曹仁字子
孝，曹洪字子廉：二人弓马熟娴，武艺精通。操大喜，于村中调练军马。卫弘尽出家财，置
办衣甲旗幡。四方送粮食者，不计其数。

    时袁绍得操矫诏，乃聚麾下文武，引兵三万，离渤海来与曹操会盟。操作檄文以达诸
郡。檄文曰：“操等谨以大义布告天下：董卓欺天罔地，灭国弑君；秽乱宫禁，残害生灵；
狼戾不仁，罪恶充积！今奉天子密诏，大集义兵，誓欲扫清华夏，剿戮群凶。望兴义师，共
泄公愤；扶持王室，拯救黎民。檄文到日，可速奉行！”操发檄文去后，各镇诸侯皆起兵相
应：第一镇，后将军南阳太守袁术。第二镇，冀州刺史韩馥。第三镇，豫州刺史孔伷。第四
镇，兖州刺史刘岱。第五镇，河内郡太守王匡。第六镇，陈留太守张邈。第七镇，东郡太守
乔瑁。第八镇，山阳太守袁遗。第九镇，济北相鲍信。第十镇，北海太守孔融。第十一镇，
广陵太守张超。第十二镇，徐州刺史陶谦。第十三镇，西凉太守马腾。第十四镇，北平太守
公孙瓒。第十五镇，上党太守张杨。第十六镇，乌程侯长沙太守孙坚。第十七镇，祁乡侯渤
海太守袁绍。诸路军马，多少不等，有三万者，有一二万者，各领文官武将，投洛阳来。

    且说北平太守公孙瓒，统领精兵一万五千，路经德州平原县。正行之间，遥见桑树丛
中，一面黄旗，数骑来迎。瓒视之，乃刘玄德也。瓒问曰：“贤弟何故在此？”玄德曰：
“旧日蒙兄保备为平原县令，今闻大军过此，将来奉候，就请兄长入城歇马。”瓒指关、张
而问曰：“此何人也？”玄德曰：“此关羽、张飞，备结义兄弟也。”瓒曰：“乃同破黄巾
者乎？”玄德曰：“皆此二人之力。”瓒曰：“今居何职？”玄德答曰：“关羽为马弓手，
张飞为步弓手。”瓒叹曰：“如此可谓埋没英雄！今董卓作乱，天下诸侯共往诛之。贤弟可
弃此卑官，一同讨贼，力扶汉室，若何？”玄德曰：“愿往。”张飞曰：“当时若容我杀了
此贼，免有今日之事。”云长曰：“事已至此，即当收拾前去。”玄德、关、张引数骑跟公
孙瓒来，曹操接着。众诸侯亦陆续皆至，各自安营下寨，连接二百余里。操乃宰牛杀马，大
会诸侯，商议进兵之策。太守王匡曰：“今奉大义，必立盟主；众听约束，然后进兵。”操
曰：“袁本初四世三公，门多故吏，汉朝名相之裔，可为盟主。”绍再三推辞，众皆曰非本
初不可，绍方应允。次日筑台三层，遍列五方旗帜，上建白旄黄钺，兵符将印，请绍登坛。
绍整衣佩剑，慨然而上，焚香再拜。其盟曰：“汉室不幸，皇纲失统。贼臣董卓，乘衅纵
害，祸加至尊，虐流百姓。绍等惧社稷沦丧，纠合义兵，并赴国难。凡我同盟，齐心戮力，
以致臣节，必无二志。有渝此盟，俾坠其命，无克遗育。皇天后土，祖宗明灵，实皆鉴
之！”读毕歃血。众因其辞气慷慨，皆涕泗横流。歃血已罢，下坛。众扶绍升帐而坐，两行
依爵位年齿分列坐定。操行酒数巡，言曰：“今日既立盟主，各听调遣，同扶国家，勿以强
弱计较。”袁绍曰：“绍虽不才，既承公等推为盟主，有功必赏，有罪必罚。国有常刑，军
有纪律。各宜遵守，勿得违犯。”众皆曰惟命是听。绍曰：“吾弟袁术总督粮草，应付诸
营，无使有缺。更须一人为先锋，直抵汜水关挑战。余各据险要，以为接应。”

    长沙太守孙坚出曰：“坚愿为前部。”绍曰：“文台勇烈，可当此任。”坚遂引本部人
马杀奔汜水关来。守关将士，差流星马往洛阳丞相府告急。董卓自专大权之后，每日饮宴。
李儒接得告急文书，径来禀卓。卓大惊，急聚众将商议。温侯吕布挺身出曰：“父亲勿虑。
关外诸侯，布视之如草芥；愿提虎狼之师，尽斩其首，悬于都门。”卓大喜曰：“吾有奉
先，高枕无忧矣！”言未绝，吕布背后一人高声出曰：“割鸡焉用牛刀？不劳温侯亲往。吾
斩众诸侯首级，如探囊取物耳！”卓视之，其人身长九尺，虎体狼腰，豹头猿臂；关西人
也，姓华，名雄。卓闻言大喜，加为骁骑校尉。拨马步军五万，同李肃、胡轸、赵岑星夜赴
关迎敌。

    众诸侯内有济北相鲍信，寻思孙坚既为前部，怕他夺了头功，暗拨其弟鲍忠，先将马步
军三千，径抄小路，直到关下搦战。华雄引铁骑五百，飞下关来，大喝：“贼将休走！”鲍
忠急待退，被华雄手起刀落，斩于马下，生擒将校极多。华雄遣人赍鲍忠首级来相府报捷，
卓加雄为都督。

    却说孙坚引四将直至关前。那四将？——第一个，右北平土垠人，姓程，名普，字德
谋，使一条铁脊蛇矛；第二个，姓黄，名盖，字公覆，零陵人也，使铁鞭；第三个，姓韩，
名当，字义公，辽西令支人也，使一口大刀；第四个，姓祖，名茂，字大荣，吴郡富春人
也，使双刀。孙坚披烂银铠，裹赤帻，横古锭刀，骑花鬃马，指关上而骂曰：“助恶匹夫，
何不早降！”华雄副将胡轸引兵五千出关迎战。程普飞马挺矛，直取胡轸。斗不数合，程普
刺中胡轸咽喉，死于马下。坚挥军直杀至关前，关上矢石如雨。孙坚引兵回至梁东屯住，使
人于袁绍处报捷，就于袁术处催粮。

    或说术曰：“孙坚乃江东猛虎；若打破洛阳，杀了董卓，正是除狼而得虎也。今不与
粮，彼军必散。”术听之，不发粮草。孙坚军缺食，军中自乱，细作报上关来。李肃为华雄
谋曰：“今夜我引一军从小路下关，袭孙坚寨后，将军击其前寨，坚可擒矣。”雄从之，传
令军士饱餐，乘夜下关。是夜月白风清。到坚寨时，已是半夜，鼓噪直进。坚慌忙披挂上
马，正遇华雄。两马相交，斗不数合，后面李肃军到，竟天价放起火来。坚军乱窜。众将各
自混战，止有祖茂跟定孙坚，突围而走。背后华雄追来。坚取箭，连放两箭，皆被华雄躲
过。再放第三箭时，因用力太猛，拽折了鹊画弓，只得弃弓纵马而奔。祖茂曰：“主公头上
赤帻射目，为贼所识认。可脱帻与某戴之。”坚就脱帻换茂盔，分两路而走。雄军只望赤帻
者追赶，坚乃从小路得脱。祖茂被华雄追急，将赤帻挂于人家烧不尽的庭柱上，却入树林潜
躲。华雄军于月下遥见赤帻，四面围定，不敢近前。用箭射之，方知是计，遂向前取了赤
帻。祖茂于林后杀出，挥双刀欲劈华雄；雄大喝一声，将祖茂一刀砍于马下。杀至天明，雄
方引兵上关。

    程普、黄盖、韩当都来寻见孙坚，再收拾军马屯扎。坚为折了祖茂，伤感不已，星夜遣
人报知袁绍。绍大惊曰：“不想孙文台败于华雄之手！”便聚众诸侯商议。众人都到，只有
公孙瓒后至，绍请入帐列坐。绍曰：“前日鲍将军之弟不遵调遣，擅自进兵，杀身丧命，折
了许多军士；今者孙文台又败于华雄：挫动锐气，为之奈何？”诸侯并皆不语。绍举目遍
视，见公孙瓒背后立着三人，容貌异常，都在那里冷笑。绍问曰：“公孙太守背后何人？”
瓒呼玄德出曰：“此吾自幼同舍兄弟，平原令刘备是也。”曹操曰：“莫非破黄巾刘玄德
乎？”瓒曰：“然。”即令刘玄德拜见。瓒将玄德功劳，并其出身，细说一遍。绍曰：“既
是汉室宗派，取坐来。”命坐。备逊谢。绍曰：“吾非敬汝名爵，吾敬汝是帝室之胄耳。”
玄德乃坐于末位，关、张叉手侍立于后。忽探子来报：“华雄引铁骑下关，用长竿挑着孙太
守赤帻，来寨前大骂搦战。”绍曰：“谁敢去战？”袁术背后转出骁将俞涉曰：“小将愿
往。”绍喜，便著俞涉出马。即时报来：“俞涉与华雄战不三合，被华雄斩了。”众大惊。
太守韩馥曰：“吾有上将潘凤，可斩华雄。”绍急令出战。潘凤手提大斧上马。去不多时，
飞马来报：“潘凤又被华雄斩了。”众皆失色。绍曰：“可惜吾上将颜良、文丑未至！得一
人在此，何惧华雄！”言未毕，阶下一人大呼出曰：“小将愿往斩华雄头，献于帐下！”众
视之，见其人身长九尺，髯长二尺，丹凤眼，卧蚕眉，面如重枣，声如巨钟，立于帐前。绍
问何人。公孙瓒曰：“此刘玄德之弟关羽也。”绍问现居何职。瓒曰：“跟随刘玄德充马弓
手。”帐上袁术大喝曰：“汝欺吾众诸侯无大将耶？量一弓手，安敢乱言！与我打出！”曹
操急止之曰：“公路息怒。此人既出大言，必有勇略；试教出马，如其不胜，责之未迟。”
袁绍曰：“使一弓手出战，必被华雄所笑。”操曰：“此人仪表不俗，华雄安知他是弓
手？”关公曰：“如不胜，请斩某头。”操教酾热酒一杯，与关公饮了上马。关公曰：“酒
且斟下，某去便来。”出帐提刀，飞身上马。众诸侯听得关外鼓声大振，喊声大举，如天摧
地塌，岳撼山崩，众皆失惊。正欲探听，鸾铃响处，马到中军，云长提华雄之头，掷于地
上。其酒尚温。后人有诗赞之曰：“威镇乾坤第一功，辕门画鼓响冬冬。云长停盏施英勇，
酒尚温时斩华雄。”曹操大喜。只见玄德背后转出张飞，高声大叫：“俺哥哥斩了华雄，不
就这里杀入关去，活拿董卓，更待何时！”袁术大怒，喝曰：“俺大臣尚自谦让，量一县令
手下小卒，安敢在此耀武扬威！都与赶出帐去！”曹操曰：“得功者赏，何计贵贱乎？”袁
术曰：“既然公等只重一县令，我当告退。”操曰：“岂可因一言而误大事耶？”命公孙瓒
且带玄德、关、张回寨。众官皆散。曹操暗使人赍牛酒抚慰三人。却说华雄手下败军，报上
关来。李肃慌忙写告急文书，申闻董卓。卓急聚李儒、吕布等商议。儒曰：“今失了上将华
雄，贼势浩大。袁绍为盟主，绍叔袁隗，现为太傅；倘或里应外合，深为不便，可先除之。
请丞相亲领大军，分拨剿捕。”卓然其说，唤李催、郭汜领兵五百，围住太傅袁隗家，不分
老幼，尽皆诛绝，先将袁隗首级去关前号令。

    卓遂起兵二十万，分为两路而来：一路先令李傕、郭汜引兵五万，把住汜水关，不要厮
杀；卓自将十五万，同李儒、吕布、樊稠、张济等守虎牢关。这关离洛阳五十里。军马到
关，卓令吕布领三万军，去关前扎住大寨。卓自在关上屯住。

    流星马探听得，报入袁绍大寨里来。绍聚众商议。操曰：“董卓屯兵虎牢，截俺诸侯中
路，今可勒兵一半迎敌。”绍乃分王匡、乔瑁、鲍信、袁遗、孙融、张杨、陶谦、公孙瓒八
路诸侯，往虎牢关迎敌。操引军往来救应。八路诸侯，各自起兵。河内太守王匡，引兵先
到。吕布带铁骑三千，飞奔来迎。王匡将军马列成阵势，勒马门旗下看时，见吕布出阵：头
戴三叉束发紫金冠，体挂西川红锦百花袍，身披兽面吞头连环铠，腰系勒甲玲珑狮蛮带；弓
箭随身，手持画戟，坐下嘶风赤兔马：果然是“人中吕布，马中赤兔”！王匡回头问曰：
“谁敢出战？”后面一将，纵马挺枪而出。匡视之，乃河内名将方悦。两马相交，无五合，
被吕布一戟刺于马下，挺戟直冲过来。匡军大败，四散奔走。布东西冲杀，如入无人之境。
幸得乔瑁、袁遗两军皆至，来救王匡，吕布方退。三路诸侯，各折了些人马，退三十里下
寨。随后五路军马都至，一处商议，言吕布英雄，无人可敌。

    正虑间，小校报来：“吕布搦战。”八路诸侯，一齐上马。军分八队，布在高冈。遥望
吕布一簇军马，绣旗招飐，先来冲阵。上党太守张杨部将穆顺，出马挺枪迎战，被吕布手起
一戟，刺于马下。众大惊。北海太守孔融部将武安国，使铁锤飞马而出。吕布挥戟拍马来
迎。战到十余合，一戟砍断安国手腕，弃锤于地而走。八路军兵齐出，救了武安国。吕布退
回去了。众诸侯回寨商议。曹操曰：“吕布英勇无敌，可会十八路诸侯，共议良策。若擒了
吕布，董卓易诛耳。”

    正议间，吕布复引兵搦战。八路诸侯齐出。公孙瓒挥槊亲战吕布。战不数合，瓒败走。
吕布纵赤兔马赶来。那马日行千里，飞走如风。看看赶上，布举画戟望瓒后心便刺。傍边一
将，圆睁环眼，倒竖虎须，挺丈八蛇矛，飞马大叫：“三姓家奴休走！燕人张飞在此！”吕
布见了，弃了公孙瓒，便战张飞。飞抖擞精神，酣战吕布。连斗五十余合，不分胜负。云长
见了，把马一拍，舞八十二斤青龙偃月刀，来夹攻吕布。三匹马丁字儿厮杀。战到三十合，
战不倒吕布。刘玄德掣双股剑，骤黄鬃马，刺斜里也来助战。这三个围住吕布。转灯儿般厮
杀。八路人马，都看得呆了。吕布架隔遮拦不定，看着玄德面上，虚刺一戟，玄德急闪。吕
布荡开阵角，倒拖画戟，飞马便回。三个那里肯舍，拍马赶来。八路军兵，喊声大震，一齐
掩杀。吕布军马望关上奔走；玄德、关、张随后赶来。古人曾有篇言语，单道着玄德、关、
张三战吕布：“汉朝天数当桓灵，炎炎红日将西倾。奸臣董卓废少帝，刘协懦弱魂梦惊。曹
操传檄告天下，诸侯奋怒皆兴兵。议立袁绍作盟主，誓扶王室定太平。温侯吕布世无比，雄
才四海夸英伟。护躯银铠砌龙鳞，束发金冠簪雉尾。参差宝带兽平吞，错落锦袍飞凤起。龙
驹跳踏起天风，画戟荧煌射秋水。出关搦战谁敢当？诸侯胆裂心惶惶。踊出燕人张冀德，手
持蛇矛丈八枪。虎须倒竖翻金线，环眼圆睁起电光。酣战未能分胜败，阵前恼起关云长。青
龙宝刀灿霜雪，鹦鹉战袍飞蛱蝶。马蹄到处鬼神嚎，目前一怒应流血。枭雄玄德掣双锋，抖
擞天威施勇烈。三人围绕战多时，遮拦架隔无休歇。喊声震动天地翻，杀气迷漫牛斗寒。吕
布力穷寻走路，遥望家山拍马还。倒拖画杆方天戟，乱散销金五彩幡。顿断绒绦走赤兔，翻
身飞上虎牢关。”三人直赶吕布到关下，看见关上西风飘动青罗伞盖。张飞大叫：“此必董
卓！追吕布有甚强处？不如先拿董贼，便是斩草除根！”拍马上关，来擒董卓。正是：擒贼
定须擒贼首，奇功端的待奇人。未知胜负如何，且听下文分解。"""

In [25]:
#test 5
word_wash(text_zh, api_data, stop_word='default')

['却说',
 '陈',
 '宫',
 '欲',
 '下手',
 '杀',
 '曹操',
 '忽',
 '转念',
 '曰',
 '国家',
 '杀',
 '不义',
 '弃',
 '插',
 '剑',
 '上马',
 '不等',
 '天明',
 '投',
 '东',
 '郡',
 '操',
 '觉',
 '不见',
 '陈',
 '宫',
 '寻思',
 '此人',
 '见',
 '说',
 '两',
 '句',
 '疑',
 '不仁',
 '弃',
 '吾',
 '急行',
 '不可',
 '久留',
 '遂',
 '连夜',
 '陈',
 '留',
 '寻',
 '见',
 '父亲',
 '备',
 '说',
 '前',
 '事',
 '欲',
 '散',
 '家',
 '资',
 '招募',
 '义兵',
 '父',
 '言',
 '资',
 '少',
 '恐',
 '事',
 '孝廉',
 '卫',
 '弘',
 '疏财仗义',
 '家',
 '巨富',
 '相助',
 '事',
 '图',
 '操',
 '置酒',
 '张',
 '筵',
 '拜请',
 '卫',
 '弘',
 '家',
 '告',
 '曰',
 '汉室',
 '主',
 '董卓',
 '专权',
 '欺君',
 '害',
 '民',
 '天下',
 '切齿',
 '操',
 '欲',
 '力',
 '扶',
 '社稷',
 '恨',
 '力',
 '不足',
 '公',
 '忠义',
 '士',
 '敢',
 '求',
 '相助',
 '卫',
 '弘',
 '曰',
 '吾',
 '心',
 '久',
 '恨',
 '未',
 '遇',
 '英雄',
 '耳',
 '孟',
 '德有',
 '大志',
 '愿',
 '家',
 '资',
 '相助',
 '操',
 '大喜',
 '先',
 '发',
 '矫诏',
 '驰',
 '报',
 '道',
 '招集',
 '义兵',
 '竖起',
 '招兵',
 '白旗',
 '一面',
 '上书',
 '忠义',
 '二',
 '字',
 '数',
 '日间',
 '应募',
 '士',
 '如雨骈集',
 '一日',
 '一个',
 '阳平',
 '卫',
 '国人',
 '姓',
 '乐',
 '名',
 '进',
 '字',
 '文

In [26]:
#test 6
cal_frequency(text_zh, api_data, stop_word='default')

[('曰', 63),
 ('绍', 34),
 ('吕布', 34),
 ('雄', 33),
 ('华', 28),
 ('关', 26),
 ('袁', 23),
 ('坚', 22),
 ('玄德', 22),
 ('诸侯', 21),
 ('太守', 21),
 ('张', 20),
 ('马', 19),
 ('镇', 19),
 ('操', 18),
 ('众', 18),
 ('皆', 16),
 ('引', 16),
 ('孙', 16),
 ('公', 14),
 ('瓒', 14),
 ('军', 14),
 ('曹操', 13),
 ('董卓', 13),
 ('飞', 12),
 ('三', 12),
 ('卓', 11),
 ('杀', 10),
 ('吾', 10),
 ('前', 10),
 ('李', 10),
 ('兵', 10),
 ('出', 10),
 ('斩', 10),
 ('战', 10),
 ('术', 9),
 ('匡', 9),
 ('路', 9),
 ('见', 8),
 ('字', 8),
 ('数', 8),
 ('姓', 8),
 ('刘', 8),
 ('孙瓒', 8),
 ('长', 8),
 ('寨', 8),
 ('茂', 8),
 ('合', 8),
 ('两', 7),
 ('言', 7),
 ('先', 7),
 ('报', 7),
 ('侯', 7),
 ('曹', 7),
 ('军马', 7),
 ('王', 7),
 ('鲍', 7),
 ('商议', 7),
 ('祖', 7),
 ('赤帻', 7),
 ('便', 7),
 ('欲', 6),
 ('弃', 6),
 ('家', 6),
 ('敢', 6),
 ('愿', 6),
 ('名', 6),
 ('时', 6),
 ('黄', 6),
 ('视', 6),
 ('问', 6),
 ('手', 6),
 ('必', 6),
 ('盟主', 6),
 ('酒', 6),
 ('急', 6),
 ('背后', 6),
 ('走', 6),
 ('刺', 6),
 ('头', 6),
 ('戟', 6),
 ('郡', 5),
 ('事', 5),
 ('卫', 5),
 ('力', 5),
 ('扶', 5),
 ('未', 

In [28]:
#test 7
check_limit(text_zh)

['却说陈宫临欲下手杀曹操，忽转念曰：“我为国家跟他到此，杀之不义。不若弃而他 往。”插剑上马，不等天明，自投东郡去了。操觉，不见陈宫，寻思：“此人见我说了这两 句，疑我不仁，弃我而去；吾当急行，不可久留。”遂连夜到陈留，寻见父亲，备说前事； 欲散家资，招募义兵。父言：“资少恐不成事。此间有孝廉卫弘，疏财仗义，其家巨富；若 得相助，事可图矣。”操置酒张筵，拜请卫弘到家，告曰：“今汉室无主，董卓专权，欺君 害民，天下切齿。操欲力扶社稷，恨力不足。公乃忠义之士，敢求相助！”卫弘曰：“吾有 是心久矣，恨未遇英雄耳。既孟德有大志，愿将家资相助。”操大喜；于是先发矫诏，驰报 各道，然后招集义兵，竖起招兵白旗一面，上书“忠义”二字。不数日间，应募之士，如雨 骈集。      一日，有一个阳平卫国人，姓乐，名进，字文谦，来投曹操。又有一个山阳巨鹿人，姓 李，名典，字曼成，也来投曹操。操皆留为帐前吏。又有沛国谯人夏侯惇，字元让，乃夏侯 婴之后；自小习枪棒；年十四从师学武，有人辱骂其师，惇杀之，逃于外方；闻知曹操起 兵，与其族弟夏侯渊两个，各引壮士千人来会。此二人本操之弟兄：操父曹嵩原是夏侯氏之 子，过房与曹家，因此是同族。不数日，曹氏兄弟曹仁、曹洪各引兵千余来助。曹仁字子 孝，曹洪字子廉：二人弓马熟娴，武艺精通。操大喜，于村中调练军马。卫弘尽出家财，置 办衣甲旗幡。四方送粮食者，不计其数。      时袁绍得操矫诏，乃聚麾下文武，引兵三万，离渤海来与曹操会盟。操作檄文以达诸 郡。檄文曰：“操等谨以大义布告天下：董卓欺天罔地，灭国弑君；秽乱宫禁，残害生灵； 狼戾不仁，罪恶充积！今奉天子密诏，大集义兵，誓欲扫清华夏，剿戮群凶。望兴义师，共 泄公愤；扶持王室，拯救黎民。檄文到日，可速奉行！”操发檄文去后，各镇诸侯皆起兵相 应：第一镇，后将军南阳太守袁术。第二镇，冀州刺史韩馥。第三镇，豫州刺史孔伷。第四 镇，兖州刺史刘岱。第五镇，河内郡太守王匡。第六镇，陈留太守张邈。第七镇，东郡太守 乔瑁。第八镇，山阳太守袁遗。第九镇，济北相鲍信。第十镇，北海太守孔融。第十一镇， 广陵太守张超。第十二镇，徐州刺史陶谦。第十三镇，西凉太守马腾。第十四镇，北平太守 公孙瓒。第十五镇，上党太守张杨。第十六镇，乌程侯长沙太守孙坚。第十七镇，祁乡侯渤 海太守袁绍。诸路军马，多少不等，有三万者，有一二万者，各领文官武

In [45]:
#test 8
word_viz(text_zh,
             '',
             api_data,
             15)

In [47]:
#test 9
word_viz(text_zh,
             '',
             api_data,
             15,
        cloud_set = WordCloud(background_color='white',font_path=r"C:\Windows\Fonts\simsun.ttc"))

In [5]:
import numpy as np
from scipy.spatial.distance import pdist
def cal_simi(text1, text2, api_data_class, size='sen', method='euc'):
    token_1 = api_data_class.get_token(text1)
    token_2 = api_data_class.get_token(text2)
    if size == 'sen':
        vec_1 = api_data_class.get_vec(text1)['documentEmbedding']
        vec_2 = api_data_class.get_vec(text2)['documentEmbedding']
    elif size == 'word':
        vec_1 = api_data_class.get_vec(text1)['tokenEmbeddings']
        vec_2 = api_data_class.get_vec(text2)['tokenEmbeddings']
        #padding
        max_matrix_length = max(len(vec_1),len(vec_2))
        add = np.zeros(300)
        if max_matrix_length == len(vec_1):
            vec_2.extend([list(add)]* (max_matrix_length-len(vec_2)))
            vec_2 = np.array(vec_2).flatten()
            vec_1 = np.array(vec_1).flatten()
        else:
            vec_1.extend([list(add)]* (max_matrix_length-len(vec_1)))
            vec_1 = np.array(vec_1).flatten()
            vec_2 = np.array(vec_2).flatten()
    else:
        print('There is no size '.format(size))

    if method == 'euc':
        score = simi_cal_euc(vec_1, vec_2, size)
    elif method == 'cos':
        score = simi_cal_cos(vec_1, vec_2, size)
    elif method == 'jac':
        score = simi_cal_jac(token_1, token_2, size)
    elif method == 'cheb':
        score = simi_cal_cheb(vec_1, vec_2, size)
    elif method == 'mah':
        score = simi_cal_mah(vec_1, vec_2, size)
    else:
        print('There is no method '.format(method))
        
    return score
        
def simi_cal_euc(vec1,vec2,size):
    if size=='word':
        pass
    else:
        vec1=np.array(vec1)
        vec2=np.array(vec2)
    return float(np.sqrt(np.sum(np.square(vec1-vec2))))
    
def simi_cal_cos(vec1,vec2,size):
    if size=='word':
        pass
    else:
        vec1=np.array(vec1)
        vec2=np.array(vec2)
    return float(np.dot(vec1,vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2)))
    
def simi_cal_mah(vec1, vec2, size):
    if size=='word':
        pass
    else:
        vec1=np.array(vec1)
        vec2=np.array(vec2)
    return float(np.sum(np.abs(vec1-vec2)))
    
def simi_cal_jac(token_1, token_2, size):
    inter = len(list(set(token_1).intersection(token_2)))
    union = (len(token_1) + len(token_1)) - inter
    return float(inter) / union

def simi_cal_cheb(vec1, vec2, size):
    if size == 'word':
        pass
    else:
        vec1=np.mat(vec1)
        vec2=np.mat(vec2)
    return float(np.max(np.abs(vec1-vec2)))

In [9]:
api_data = Data()

In [7]:
text1 = 'the cat ate the mouse'
text2 = 'the mouse ate by the cat'

In [62]:
#test 10
print(cal_simi(text1, text2, api_data, 'sen', 'euc'))

6.244997997916865e-08


In [8]:
#test 11
print(cal_simi(text1, text2, api_data, 'word', 'euc'))

2.169246985916903


In [65]:
#test 12
print(cal_simi(text1, text2, api_data, 'sen', 'cos'))

0.9999999999999981


In [10]:
#test 13
print(cal_simi(text1, text2, api_data, 'word', 'cos'))

0.5745945236631423


In [80]:
#test 14
print(cal_simi(text1, text2, api_data, 'sen', 'jac'))

0.6666666666666666


In [84]:
#text 15
print(cal_simi(text1, text2, api_data, 'sen', 'cheb'))

2.000000000335067e-08


In [86]:
#test 16
print(cal_simi(text1, text2, api_data, 'sen', 'mah'))

3.099999999773423e-07
